# Global data views from CDS

Trying to create a better version of this: https://www.linkedin.com/posts/copernicus-ecmwf_wildfire-saharandust-copernicusatmosphere-ugcPost-7221877806152970243-nVmb?utm_source=share&utm_medium=member_desktop

In [8]:
from __future__ import annotations

import numpy as np

import geovista as gv
from geovista.samples import regular_grid
import geovista.theme

import pyvista as pv

import os.path

import cdsapi

import xarray as xr

import pvxarray

import yaml


In [9]:

c = cdsapi.Client()

In [12]:
with open('adsapi.yml', 'r') as f:
            credentials = yaml.safe_load(f)

c = cdsapi.Client(url=credentials['url'], key=credentials['key'])

c.retrieve(
    'cams-global-atmospheric-composition-forecasts',
    {
        'date': '2024-08-06/2024-08-06',
        'type': 'forecast',
        'format': 'netcdf_zip',
        'variable': [
            '10m_u_component_of_wind', '10m_v_component_of_wind', 'total_aerosol_optical_depth_550nm',
        ],
        'time': '00:00',
        'leadtime_hour': [
            '0', '1', '10',
            '11', '12', '13',
            '14', '15', '16',
            '17', '18', '19',
            '2', '20', '21',
            '22', '23', '3',
            '4', '5', '6',
            '7', '8', '9',
        ],
    },
    'download.netcdf_zip')

2024-08-06 21:05:23,671 INFO Welcome to the CDS
2024-08-06 21:05:23,671 INFO Sending request to https://ads.atmosphere.copernicus.eu/api/v2/resources/cams-global-atmospheric-composition-forecasts
2024-08-06 21:05:23,738 INFO Request is queued
2024-08-06 21:05:24,776 INFO Request is running
2024-08-06 21:05:28,640 INFO Request is completed
2024-08-06 21:05:28,640 INFO Downloading https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.mars_constrained.internal-1722974727.8403738-2077-12-7d8d2d44-b8c0-4055-8c4d-f018dd4f3665.zip to download.netcdf_zip (55.7M)
2024-08-06 21:05:38,443 INFO Download rate 5.7M/s   


Result(content_length=58456836,content_type=application/zip,location=https://download-0001-ads-clone.copernicus-climate.eu/cache-compute-0001/cache/data0/adaptor.mars_constrained.internal-1722974727.8403738-2077-12-7d8d2d44-b8c0-4055-8c4d-f018dd4f3665.zip)

In [16]:
xr_era5 = xr.open_mfdataset("data.nc",engine="netcdf4")

In [17]:
xr_era5

<xarray.Dataset>
Dimensions:    (longitude: 900, latitude: 451, time: 24)
Coordinates:
  * longitude  (longitude) float32 0.0 0.4 0.8 1.2 ... 358.4 358.8 359.2 359.6
  * latitude   (latitude) float32 90.0 89.6 89.2 88.8 ... -89.2 -89.6 -90.0
  * time       (time) datetime64[ns] 2024-08-06 ... 2024-08-06T23:00:00
Data variables:
    v10        (time, latitude, longitude) float32 dask.array<chunksize=(24, 451, 900), meta=np.ndarray>
    aod550     (time, latitude, longitude) float32 dask.array<chunksize=(24, 451, 900), meta=np.ndarray>
    u10        (time, latitude, longitude) float32 dask.array<chunksize=(24, 451, 900), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-08-06 20:05:27 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [ ]:
# Get wind and solar data for a week


year = 2024
month = 1
day = 1

for day in range(1,8):
    outfile = f"data/era5_{year}{month:02}{day:02}.grib"

    if not os.path.isfile(outfile):

        c = cdsapi.Client()

        cds_dataset = "reanalysis-era5-single-levels"
        cds_request = {
            "product_type": "reanalysis",
            "format": "grib",
            "variable": [
                "100m_u_component_of_wind", "100m_v_component_of_wind", "surface_solar_radiation_downwards",
            ],
            "year": "2023",
            "month": "12",
            "day":"1",
            'time': [
                    '00:00', '01:00', '02:00',
                    '03:00', '04:00', '05:00',
                    '06:00', '07:00', '08:00',
                    '09:00', '10:00', '11:00',
                    '12:00', '13:00', '14:00',
                    '15:00', '16:00', '17:00',
                    '18:00', '19:00', '20:00',
                    '21:00', '22:00', '23:00',
                ],
        }

        cds_request.update({"year":str(year), "month":str(month), "day":str(day)})

        print(cds_request)

        c.retrieve(cds_dataset, cds_request, outfile)


In [18]:
xr_era5 = xr.open_mfdataset("data/era5*.nc")

In [19]:
step = 5

xr_era5_coarse_time = xr_era5.isel(longitude=np.arange(0,len(xr_era5.longitude),step=step),latitude=np.arange(0,len(xr_era5.latitude),step=step))

In [20]:
xr_era5_coarse_time

<xarray.Dataset>
Dimensions:     (valid_time: 168, latitude: 145, longitude: 288)
Coordinates:
    number      int64 0
  * valid_time  (valid_time) datetime64[ns] 2024-01-01 ... 2024-01-07T23:00:00
  * latitude    (latitude) float64 90.0 88.75 87.5 86.25 ... -87.5 -88.75 -90.0
  * longitude   (longitude) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
    expver      (valid_time) <U4 dask.array<chunksize=(24,), meta=np.ndarray>
Data variables:
    u100        (valid_time, latitude, longitude) float32 dask.array<chunksize=(2, 10, 36), meta=np.ndarray>
    v100        (valid_time, latitude, longitude) float32 dask.array<chunksize=(2, 10, 36), meta=np.ndarray>
    ssrd        (valid_time, latitude, longitude) float32 dask.array<chunksize=(2, 10, 36), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-05T21:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [21]:
xr_era5_coarse = xr_era5_coarse_time.isel(valid_time=8)

In [22]:
xr_era5_coarse

<xarray.Dataset>
Dimensions:     (latitude: 145, longitude: 288)
Coordinates:
    number      int64 0
    valid_time  datetime64[ns] 2024-01-01T08:00:00
  * latitude    (latitude) float64 90.0 88.75 87.5 86.25 ... -87.5 -88.75 -90.0
  * longitude   (longitude) float64 0.0 1.25 2.5 3.75 ... 356.2 357.5 358.8
    expver      <U4 dask.array<chunksize=(), meta=np.ndarray>
Data variables:
    u100        (latitude, longitude) float32 dask.array<chunksize=(10, 36), meta=np.ndarray>
    v100        (latitude, longitude) float32 dask.array<chunksize=(10, 36), meta=np.ndarray>
    ssrd        (latitude, longitude) float32 dask.array<chunksize=(10, 36), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-08-05T21:34 GRIB to CDM+CF via cfgrib-0.9.1...

In [23]:
# xr_era5_coarse = xr_era5_coarse_time.isel(valid_time=8)

x_sol = xr_era5_coarse["longitude"].values
y_sol = xr_era5_coarse["latitude"].values

z_sol = xr_era5_coarse["ssrd"].values
z_sol = z_sol/z_sol.max()

x_sol = np.hstack([x_sol,x_sol[0]])
# y_sol = np.hstack([y_sol,y_sol[0]])
z_sol = np.vstack([z_sol.T,z_sol[:,0]]).T

#mesh_solar = gv.Transform.from_1d(x_sol, y_sol, data=z_sol)
mesh_solar = gv.Transform.from_1d(x_sol, y_sol, data=z_sol)


In [24]:
print(x_sol.shape)
print(y_sol.shape)
print(z_sol.shape)

(289,)
(145,)
(145, 289)


In [25]:
class SetVisibilityCallback:
    """Helper callback to keep a reference to the actor being modified."""

    def __init__(self, actor):
        self.actor = actor

    def __call__(self, state):
        self.actor.SetVisibility(state)

In [26]:
x_wnd = xr_era5_coarse["longitude"].values
y_wnd = xr_era5_coarse["latitude"].values

u_wnd = xr_era5_coarse["u100"].values.flatten()
v_wnd = xr_era5_coarse["v100"].values.flatten()

mesh_wnd = gv.Transform.from_1d(x_wnd, y_wnd)
mesh_wnd["vectors"] = np.vstack([u_wnd,v_wnd]).T

In [27]:
# Generate a meshgrid.
lon,lat = np.meshgrid(xr_era5_coarse["longitude"],xr_era5_coarse["latitude"])

# convert to 3D sphere
lon = lon.flatten()*np.pi/180
lat = lat.flatten()*np.pi/180
radius = 1.1

x = np.sin(np.pi/2-lat) * np.cos(lon)
y = np.sin(np.pi/2-lat) * np.sin(lon)
z = np.cos(np.pi/2-lat)

mesh_array = radius*np.vstack([x,y,z]).T

mesh = pv.PolyData(mesh_array)

u100 = xr_era5_coarse["u100"].values.flatten()
v100 = xr_era5_coarse["v100"].values.flatten()

x100 = u100*np.sin(lon)+v100*np.sin(lat)*np.cos(lon)
y100 = -u100*np.sin(lon+np.pi/2)+v100*np.sin(lat)*np.sin(lon)
z100 = -v100*np.cos(lat)

scale_vector = 0.002

mesh["vectors"] = np.vstack((x100,y100,z100)).T * scale_vector
mesh.set_active_vectors("vectors")


In [28]:
plotter = gv.GeoPlotter(off_screen=True, window_size=[512,384])#,image_scale=0.5)

# optimize for size
plotter.open_gif('Map31-Favourite-WindSolarGlobe.gif', palettesize=16, subrectangles=True)

# initiate image and static values
plotter.add_base_layer(texture=gv.natural_earth_1(), radius=.999, lighting=False)
plotter.add_coastlines(color="white", radius=0.99999)

plotter.camera.zoom(1.7)

xr_era5_coarse = xr_era5_coarse_time.isel(valid_time=1)
x_sol = xr_era5_coarse["longitude"].values
y_sol = xr_era5_coarse["latitude"].values

z_sol = xr_era5_coarse["ssrd"].values
z_sol = z_sol/z_sol.max()

x_sol = np.hstack([x_sol,x_sol[0]])
z_sol = np.vstack([z_sol.T,z_sol[:,0]]).T

# clear and overwrite the mesh on each frame
n_frames = 24
for day in range(n_frames):
    xr_era5_coarse = xr_era5_coarse_time.isel(valid_time=day)

    z_sol = xr_era5_coarse["ssrd"].values
    z_sol = z_sol/z_sol.max()
    z_sol = np.vstack([z_sol.T,z_sol[:,0]]).T

    mesh_solar = gv.Transform.from_1d(x_sol, y_sol, data=z_sol)

    mesh_wnd = gv.Transform.from_1d(x_wnd, y_wnd)

    # Generate a meshgrid.
    lon,lat = np.meshgrid(xr_era5_coarse["longitude"],xr_era5_coarse["latitude"])

    # convert to 3D sphere
    lon = lon.flatten()*np.pi/180
    lat = lat.flatten()*np.pi/180

    u100 = xr_era5_coarse["u100"].values.flatten()
    v100 = xr_era5_coarse["v100"].values.flatten()

    x100 = u100*np.sin(lon)+v100*np.sin(lat)*np.cos(lon)
    y100 = -u100*np.sin(lon+np.pi/2)+v100*np.sin(lat)*np.sin(lon)
    z100 = -v100*np.cos(lat)

    mesh_wnd["vectors"] = np.vstack((x100,y100,z100)).T * scale_vector
    mesh_wnd.set_active_vectors("vectors")

    # Plot the vectors.    
    actor_solar = plotter.add_mesh(mesh_solar, radius=0.9999, cmap="cividis", opacity="linear_r", lighting=False, label='Sun', show_scalar_bar=False)

    actor_wind = plotter.add_mesh(mesh_wnd.arrows, radius=0.999999, lighting=False, label='Wind', show_scalar_bar=False)

    actor_text = plotter.add_text(
        str(xr_era5_coarse.valid_time.values)[:13],
        position="upper_left",
        font_size=10,
        shadow=True,
    )

    plotter.write_frame()  # Write this frame

    plotter.remove_actor(actor_solar)
    plotter.remove_actor(actor_wind)
    plotter.remove_actor(actor_text)

# Be sure to close the plotter when finished
plotter.close()


# plotter.add_legend(bcolor=None, face=None, size=(0.1,0.1), loc="upper right")

# plotter.add_axes()

# plotter.show()

c:\Users\Charlie\Documents\GitHub\mapping-challenge\.venv\Lib\site-packages\pyvista\core\pointset.py:1129: PyVistaDeprecationWarning: The current behavior of `pv.PolyData.n_faces` has been deprecated.
                Use `pv.PolyData.n_cells` or `pv.PolyData.n_faces_strict` instead.
                See the documentation in '`pv.PolyData.n_faces` for more information.
  warnings.warn(
c:\Users\Charlie\Documents\GitHub\mapping-challenge\.venv\Lib\site-packages\pyvista\core\dataset.py:595: PyVistaDeprecationWarning: Use of `DataSet.active_t_coords` is deprecated. Use `DataSet.active_texture_coordinates` instead.
  warnings.warn(


In [29]:
from pygifsicle import optimize
optimize("Map31-Favourite-WindSolarGlobe.gif")